In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.model_selection import train_test_split, cross_val_score 
from statistics import mean
from sklearn import preprocessing


import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

 - Resources: https://github.com/NicolasHug/Surprise

In [2]:
df_total = pd.read_csv("model_data.csv",index_col = 0)

In [3]:
df_total.head(10)

,Beer_id,Username_id,User Score
0,84627,30162,3.08
1,84627,11051,3.13
2,84627,24945,4.07
3,84627,30155,3.38
4,84627,30731,4.07
5,84627,1942,3.83
6,84627,21452,3.95
7,84627,25654,3.47
8,84627,3307,4.00
9,84627,8724,3.38


In [4]:
df_total.shape

(678042, 3)

In [5]:
df_total.columns = ["beer_id", "User_id", "ratings"]

In [6]:
df_total[df_total["User_id"] == 15186]

,beer_id,User_id,ratings
1607384,92551,15186,3.99


 - Converting the dataframe into the dataset required for susprise library

In [7]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import random

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [8]:
reader = Reader(rating_scale=(0, 5,))

In [9]:
data = Dataset.load_from_df(df_total[['User_id', 'beer_id', 'ratings']], reader)

raw_ratings = data.raw_ratings

In [10]:
random.shuffle(raw_ratings)

In [11]:
threshold = int(.8 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

In [12]:
data.raw_ratings = A_raw_ratings

 - Grid search to find the best number of epochs, regularization and factors
 ##### Takes a long time to run!

In [13]:
print('Grid Search...')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'n_factors' : [10,20,50]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)
algo = grid_search.best_estimator['rmse']
grid_search.best_estimator['rmse']





Grid Search...


In [14]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from tabulate import tabulate
from six import iteritems

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [15]:
table = [[] for _ in range(len(grid_search.cv_results['params']))]
for i in range(len(grid_search.cv_results['params'])):
    for key in grid_search.cv_results.keys():
        table[i].append(grid_search.cv_results[key][i])

header = grid_search.cv_results.keys()
print(tabulate(table, header, tablefmt="rst"))

print()

for key, val in iteritems(grid_search.cv_results):
    print('{:<20}'.format("'" + key + "':"), end='')
    if isinstance(val[0], float):
        print([float('{:.2f}'.format(f)) for f in val])
    else:
        print(val)

==================  ==================  ==================  ================  ===============  ================  ===============  ==============  ================  ===============  ==================================================  ================  ==============  =================
  split0_test_rmse    split1_test_rmse    split2_test_rmse    mean_test_rmse    std_test_rmse    rank_test_rmse    mean_fit_time    std_fit_time    mean_test_time    std_test_time  params                                                param_n_epochs    param_lr_all    param_n_factors
==================  ==================  ==================  ================  ===============  ================  ===============  ==============  ================  ===============  ==================================================  ================  ==============  =================
          0.524319            0.524384            0.521488          0.523397       0.0013501                 10          6.06293        0.256689 

In [16]:
print(grid_search.best_params['rmse'])


{'n_epochs': 10, 'lr_all': 0.005, 'n_factors': 10}


 - Training the best parameters 

In [17]:
trainset = data.build_full_trainset()
algo.fit(trainset)

In [18]:
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

Biased accuracy on A,   RMSE: 0.4370


0.4370098364195614

In [19]:
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Unbiased accuracy on B, RMSE: 0.4705


0.47047036010314747

In [23]:
predictions[0:5]

[Prediction(uid=16317, iid=92246, r_ui=4.26, est=3.8531529793036614, details={'was_impossible': False}),
 Prediction(uid=17816, iid=94392, r_ui=4.77, est=4.532574443464895, details={'was_impossible': False}),
 Prediction(uid=6684, iid=60085, r_ui=4.23, est=3.9311162590025672, details={'was_impossible': False}),
 Prediction(uid=2836, iid=16590, r_ui=4.1, est=3.941024983600738, details={'was_impossible': False}),
 Prediction(uid=26454, iid=37471, r_ui=2.6, est=3.300559795541776, details={'was_impossible': False})]

### RECOMMENDATIONS USING SURPRISE LIBRARY

In [24]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset



def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

In [25]:
top_n = get_top_n(predictions, n=10)


In [37]:
uid = str(800000)
iid = str(80468)
pred = algo.predict(uid, iid, verbose=True)

user: 800000     item: 80468      r_ui = None   est = 3.78   {'was_impossible': False}


In [36]:
top_n[0]

[(2111, 4.232251425130251),
 (19666, 4.0460323167355075),
 (3387, 4.002804044745935)]

In [38]:
# Recommendation user 4579 - same as used as example for neural network model
top_n[4579]

[(11422, 3.968342948218224)]

In [39]:
top_n[24077]

[(92294, 4.199864207514829),
 (47768, 4.111555471641415),
 (61711, 4.107482192047547)]

In [40]:
#Recommendations

In [41]:
for key in top_n:
   print("user: %s , beer_recommendation: %s" % (key, top_n[key]))

user: 16317 , beer_recommendation: [(22975, 4.178821642800412), (99191, 4.1683783651940844), (11017, 4.089490348494144), (6574, 4.084932584284562), (18731, 4.083871673544072), (7509, 4.081644266908293), (28344, 4.059863617058503), (55240, 4.051239586693948), (51825, 4.050232876542592), (84094, 4.044360977583435)]
user: 17816 , beer_recommendation: [(23567, 4.888802205322349), (72516, 4.76518153570974), (31036, 4.747114418570987), (44536, 4.74571901054559), (26509, 4.739345217801964), (31497, 4.710860396543848), (26500, 4.704079300954442), (44932, 4.701798634097685), (62897, 4.699720218274281), (63257, 4.688723807371187)]
user: 6684 , beer_recommendation: [(60085, 3.9311162590025672), (48643, 3.9099465722673123), (98257, 3.803499709644484), (42711, 3.7905452391847048), (70473, 3.7204739210008233), (48197, 3.6163863713361173)]
user: 2836 , beer_recommendation: [(81868, 3.9929162021852838), (72096, 3.9569931086903174), (26381, 3.95697036267908), (16590, 3.941024983600738), (31868, 3.94076

user: 12302 , beer_recommendation: [(64385, 2.978326467776058), (78395, 2.977630109057619), (67432, 2.9743018004133672), (58363, 2.9709651777000583), (3985, 2.9513623415459334), (49970, 2.951159459051917), (9302, 2.9454012438180106), (9916, 2.914857834858256), (66392, 2.9106735933547583), (84361, 2.907447472290009)]
user: 8907 , beer_recommendation: [(23707, 3.947840469803877), (87221, 3.825190207395423)]
user: 35100 , beer_recommendation: [(11111, 3.8779369319540393)]
user: 33903 , beer_recommendation: [(94012, 3.965462258239677), (95334, 3.9298697183450835)]
user: 15574 , beer_recommendation: [(94712, 4.3654720690642534), (48006, 4.363420822321403), (86337, 4.203557408949816), (14757, 4.189556068160889), (89008, 4.168360083332057), (47999, 4.165813098810424), (99672, 4.145709300959064), (11215, 4.102706895000824), (94801, 4.078891766746727), (98888, 4.078117716269636)]
user: 6576 , beer_recommendation: [(16336, 4.150372913720118), (100232, 4.043923702718402), (3489, 3.971173748572407

user: 7908 , beer_recommendation: [(99399, 3.999974634215007), (42067, 3.912797003824328), (76615, 3.8645936966066197), (19125, 3.8204394654750367), (58836, 3.8140544747254634), (24807, 3.761938522843918), (16749, 3.6877785317090117)]
user: 23425 , beer_recommendation: [(56777, 3.850135306095639), (91444, 3.8348704223364822), (16770, 3.7691432417390898)]
user: 14836 , beer_recommendation: [(8429, 3.9070482219883482), (57324, 3.8066125597608345)]
user: 12948 , beer_recommendation: [(34324, 3.801173793082014), (73120, 3.7111900813011585), (27997, 3.688932604588769), (36618, 3.668653856494451), (52874, 3.64981277434507), (90336, 3.5336549072286654), (92913, 3.4957489658379353), (84891, 3.491933709809459), (24548, 3.4341140779349453)]
user: 24652 , beer_recommendation: [(10973, 3.9580874932369094), (74974, 3.846347708676864), (84585, 3.7887342498275434)]
user: 15229 , beer_recommendation: [(7613, 3.847002178888702)]
user: 12504 , beer_recommendation: [(97968, 3.8754845193885212), (28667, 3

user: 11433 , beer_recommendation: [(1983, 3.9020686443887236), (1471, 3.891764709620229), (5719, 3.8544712867753557), (77497, 3.806240420328948), (66550, 3.715986682543895), (85876, 3.6753563708982613), (4382, 3.6604874234395144), (66532, 3.659622124897476), (77117, 3.6448949574615703), (91783, 3.639200417622003)]
user: 28743 , beer_recommendation: [(84149, 3.6872517130156615)]
user: 24202 , beer_recommendation: [(59705, 3.9687073212813875), (33375, 3.9245268142847025), (36985, 3.7063538146480894)]
user: 21477 , beer_recommendation: [(10062, 4.104998829115475), (25880, 4.084284416519206), (17577, 3.837168945089308)]
user: 11641 , beer_recommendation: [(46286, 3.8116677361296847), (41584, 3.6049410988708384), (19123, 3.4282871262517736), (41738, 3.29180534931813)]
user: 18989 , beer_recommendation: [(54246, 4.272837618229414), (47331, 4.259978198627087), (98854, 4.216041382229963), (53344, 4.179809976673754), (54208, 4.1393171686803445), (56172, 4.047866348748605)]
user: 2359 , beer_re

user: 27547 , beer_recommendation: [(36421, 3.972079196304661), (46693, 3.8844321113372895)]
user: 15244 , beer_recommendation: [(71814, 3.5908536570888763)]
user: 31802 , beer_recommendation: [(71184, 4.115468679199301), (94004, 3.7803001868319157), (3677, 3.7769643336448153), (12612, 3.761468789565528), (41285, 3.6924356504758222), (44303, 3.666703436308802), (78592, 3.393827832367135)]
user: 1722 , beer_recommendation: [(69949, 3.8793370573672594), (81548, 3.855454297278129), (96154, 3.851769619949756), (61083, 3.6841444876536324), (58766, 3.628287858995063), (5328, 3.294653307107625)]
user: 30196 , beer_recommendation: [(55512, 4.524726129585853), (27025, 4.372273236405252), (71605, 4.359442482500889), (13652, 4.247512890341457), (44004, 4.177803941017709), (90751, 4.154423980424851), (17262, 3.797681569754287)]
user: 7468 , beer_recommendation: [(81231, 3.8587854572439952)]
user: 35270 , beer_recommendation: [(35623, 2.7350108828875075)]
user: 14508 , beer_recommendation: [(86120,

...

user: 6910 , beer_recommendation: [(43758, 3.8086438002720198)]
user: 20852 , beer_recommendation: [(16484, 3.8056316664691545)]
user: 21189 , beer_recommendation: [(38483, 3.7175543464796905), (61073, 3.1555291179499707)]
user: 22279 , beer_recommendation: [(77501, 3.7003214221448957)]
user: 9587 , beer_recommendation: [(67327, 3.9557815242277417)]
user: 8435 , beer_recommendation: [(96168, 3.760703168498913)]
user: 13455 , beer_recommendation: [(7649, 3.773948600438759)]
user: 8249 , beer_recommendation: [(87836, 3.7614121330029042), (83619, 3.693003406886835)]
user: 19254 , beer_recommendation: [(51695, 3.5323573327227855)]
user: 25775 , beer_recommendation: [(80408, 4.026876459770878)]
user: 28812 , beer_recommendation: [(16597, 3.9659014303648448)]
user: 19322 , beer_recommendation: [(78984, 3.8418417056474103)]
user: 26826 , beer_recommendation: [(95083, 3.6962229201719365), (83004, 3.648829593701107)]
user: 35971 , beer_recommendation: [(27158, 3.9084850772801176), (31493, 3.780

user: 35271 , beer_recommendation: [(59926, 3.5915709838584635)]
user: 23209 , beer_recommendation: [(29448, 3.93031644517615)]
user: 25143 , beer_recommendation: [(93022, 3.9020975582069), (8915, 3.8387680708435177)]
user: 11301 , beer_recommendation: [(7008, 3.8041706167746248)]
user: 31100 , beer_recommendation: [(16190, 3.7533619316210696), (91047, 3.6754121969959437)]
user: 5612 , beer_recommendation: [(87789, 3.7311475556769103)]
user: 36049 , beer_recommendation: [(50238, 3.8382640770283625), (7395, 3.65320977351589)]
user: 1644 , beer_recommendation: [(12785, 3.4978467985585104)]
user: 6532 , beer_recommendation: [(97296, 3.682850940808462), (19918, 3.4566732988272157)]
user: 3664 , beer_recommendation: [(2053, 3.6702547800989893)]
user: 33688 , beer_recommendation: [(13133, 4.014604711748764)]
user: 34342 , beer_recommendation: [(64863, 4.158906596153619)]
user: 1785 , beer_recommendation: [(17935, 3.0272445262181713)]
user: 6776 , beer_recommendation: [(55362, 3.8253813437518

user: 30047 , beer_recommendation: [(9900, 3.929756656337893)]
user: 14877 , beer_recommendation: [(50350, 3.8869905873840622), (14800, 3.877724568548618), (56360, 3.8253418845792413), (28217, 3.812892942403517)]
user: 26129 , beer_recommendation: [(26208, 3.648087646083765)]
user: 20810 , beer_recommendation: [(86297, 3.58168135594306)]
user: 3568 , beer_recommendation: [(21210, 3.7307043728876037)]
user: 17055 , beer_recommendation: [(63194, 3.7491997667160306)]
user: 34526 , beer_recommendation: [(73979, 3.8874304631181884)]
user: 20793 , beer_recommendation: [(66357, 3.6523676014208686)]
user: 14395 , beer_recommendation: [(11243, 3.7850318425308425)]
user: 18401 , beer_recommendation: [(191, 3.8285149667694176)]
user: 35619 , beer_recommendation: [(4841, 3.747577778762641)]
user: 24826 , beer_recommendation: [(95685, 3.984748994884051)]
user: 5420 , beer_recommendation: [(28692, 3.835853012104208), (62715, 3.7663045766400116)]
user: 34332 , beer_recommendation: [(70091, 3.74057985

user: 30842 , beer_recommendation: [(30649, 3.7205944936203434)]
user: 29207 , beer_recommendation: [(83252, 4.1392436264955945)]
user: 7664 , beer_recommendation: [(89931, 4.093389802446684), (90031, 3.8965427905259697)]
user: 16131 , beer_recommendation: [(35823, 3.772503541711557)]
user: 4971 , beer_recommendation: [(69835, 3.8215661480168657)]
user: 35015 , beer_recommendation: [(3497, 3.8308453743052846)]
user: 12461 , beer_recommendation: [(89064, 3.8022558579886345)]
user: 26122 , beer_recommendation: [(65625, 3.773063737561045)]
user: 26133 , beer_recommendation: [(73459, 3.3922167701982304)]
user: 16094 , beer_recommendation: [(6569, 3.8039088627167814)]
user: 22690 , beer_recommendation: [(9167, 3.837665263274842)]
user: 28164 , beer_recommendation: [(536, 3.8736381217243387)]
user: 23284 , beer_recommendation: [(53996, 3.609212639010617), (47190, 3.5268909694397017)]
user: 30538 , beer_recommendation: [(47042, 3.7704875973368943)]
user: 34576 , beer_recommendation: [(77846, 

user: 7083 , beer_recommendation: [(69861, 3.7002982020266812)]
user: 32842 , beer_recommendation: [(71325, 3.865488998734472)]
user: 15734 , beer_recommendation: [(20360, 3.849349177438315)]
user: 21194 , beer_recommendation: [(98118, 4.101114157855932)]
user: 14502 , beer_recommendation: [(15828, 3.661487690296164)]
user: 7869 , beer_recommendation: [(86657, 3.8582899453397093)]
user: 12278 , beer_recommendation: [(94160, 3.7719915141649456)]
user: 9990 , beer_recommendation: [(49474, 3.1427504440884375)]
user: 17772 , beer_recommendation: [(70736, 3.957495371409304)]
user: 23279 , beer_recommendation: [(39419, 4.220550076063355)]
user: 16672 , beer_recommendation: [(38414, 3.161271642512994)]
user: 8180 , beer_recommendation: [(93104, 3.8227194519530134)]
user: 24017 , beer_recommendation: [(97775, 3.655797260676416)]
user: 19118 , beer_recommendation: [(55613, 3.825156453392528)]
user: 8121 , beer_recommendation: [(6503, 2.9916754756849433)]
user: 20685 , beer_recommendation: [(177